In [ ]:
!pip3 install pandas
!pip3 install sqlalchemy
!pip3 install psycopg2  # For PostgreSQL
!pip3 install pymysql   # For MySQL
!pip3 install cx_Oracle # For Oracle 
!pip3 install pyodbc

In [ ]:
import pyodbc
import pandas as pd

# Define your connection string
connection_string = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=Rahul\MSSQLSERVER01;"
    "DATABASE=ECC60jkl_HACK;"
    "Trusted_Connection=yes;"
)

# Establish the connection
connection = pyodbc.connect(connection_string)

# Use the connection to query the database with pandas
source = pd.read_sql("SELECT * FROM dbo.A017", connection)

# Print the dataframe
print(source)



In [ ]:
!pip3 install dask

In [ ]:
import pandas as pd
import pyodbc

# Define the connection string for the target database (Oracle EBS)
target_connection_string = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=Rahul\MSSQLSERVER01;"
    "DATABASE=ORACLE_EBS_HACK;"
    "Trusted_Connection=yes;"
)

# Establish the connection
connection = pyodbc.connect(target_connection_string)

# Use the connection to query the database with pandas
source = pd.read_sql("SELECT * FROM APPLSYS_FND_FLEX_VALUES", connection)

# Print the dataframe
print(source)

In [ ]:
# Get the schema information for the Oracle EBS database
target_cursor = target_connection.cursor()
target_cursor.execute("SELECT TABLE_NAME, COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS")
target_schema = target_cursor.fetchall()

# Get the schema information for the ECC60jkl_HACK database
source_cursor = connection.cursor()
source_cursor.execute("SELECT TABLE_NAME, COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS")
source_schema = source_cursor.fetchall()

In [ ]:
target_schema_dict = {}
for table, column, data_type in target_schema:
    print(f"Target schema: {table}.{column} ({data_type})")
    if table not in target_schema_dict:
        target_schema_dict[table] = []
    target_schema_dict[table].append((column, data_type))

print(f"Target schema dict: {target_schema_dict}")

source_schema_dict = {}
for table, column, data_type in source_schema:
    print(f"Source schema: {table}.{column} ({data_type})")
    if table not in source_schema_dict:
        source_schema_dict[table] = []
    source_schema_dict[table].append((column, data_type))

print(f"Source schema dict: {source_schema_dict}")

In [ ]:
similar_tables = {}
for target_table, target_columns in target_schema_dict.items():
    for source_table, source_columns in source_schema_dict.items():
        similarity = len(set(target_columns) & set(source_columns)) / len(set(target_columns) | set(source_columns))
        if similarity > 0.5:  # adjust the threshold as needed
            similar_tables[target_table] = source_table

print(f"Similar tables: {similar_tables}")

In [ ]:
similar_columns = {}
for target_table, target_columns in target_schema_dict.items():
    for source_table, source_columns in source_schema_dict.items():
        if target_table == source_table:  # only compare columns within the same table
            for target_column_name in [col[1] for col in target_columns]:
                for source_column_name in [col[1] for col in source_columns]:
                    if target_column_name == source_column_name:
                        similarity = 1.0  # exact match
                    else:
                        similarity = levenshtein_distance(target_column_name, source_column_name)
                    if similarity > 0.5:  # adjust the threshold as needed
                        similar_columns[(target_table, target_column_name)] = (source_table, source_column_name)

print(f"Similar columns: {similar_columns}")

In [ ]:
!pip3 install jellyfish 